In [1]:
using Oscar

  ___   ___   ___    _    ____
 / _ \ / __\ / __\  / \  |  _ \  | Combining and extending ANTIC, GAP,
| |_| |\__ \| |__  / ^ \ |  ´ /  | Polymake and Singular
 \___/ \___/ \___//_/ \_\|_|\_\  | Type "?Oscar" for more information
o--------o-----o-----o--------o  | Documentation: https://docs.oscar-system.org
  S Y M B O L I C   T O O L S    | Version 1.4.1


In [2]:
# Defined the denominator
sl2size(p, k) = p^(2*k - 2) * (p^2 - 1)

sl2size (generic function with 1 method)

In [39]:
function adder(p, n, kbig, lift_dict, a, b, c_start, exp, Rn, Rkbig)
    key = (Rn(a), Rn(b), Rn(c_start))
    if haskey(lift_dict, key)
        lifted_b = lift_dict[key][2]
        if iszero(lifted_b) || valuation(lift(lifted_b), p) == exp
            return false
        end
    end

    lift_dict[key] = (Rkbig(a), Rkbig(b), Rkbig(c_start))
    for c in (c_start + p^(kbig-exp)) : p^(kbig-exp) : (p^n-1)
        lift_dict[(Rn(a), Rn(b), Rn(c))] = (Rkbig(a), Rkbig(b), Rkbig(c))
    end
    return true
end

adder (generic function with 1 method)

In [44]:
# this function build matrices satisfying the condition of ratio2, while keeping track of the lifted form
function build_matrices_ratio2(p, kbig, n, trace, det)
    Rn, = residue_ring(ZZ, p^n)
    Rkbig, = residue_ring(ZZ, p^kbig)
    lift_dict = Dict{Tuple{zzModRingElem, zzModRingElem, zzModRingElem}, Tuple{zzModRingElem, zzModRingElem, zzModRingElem}}()

    for a in 0:p^n-1
        for b in 1:p^n-1
            if b % p != 0
                c = Rkbig(a*(trace - a) - det) / Rkbig(b)
                lift_dict[(Rn(a), Rn(b), Rn(c))] = (Rkbig(a), Rkbig(b), Rkbig(c))
            end
        end
    end
    
    for exp in 1:(n-1)
        for a in 0:(p^kbig - 1)
            rhs = Rkbig(a * (trace - a) - det)
            if !iszero(rhs)
                pval = valuation(lift(rhs), p)
                if pval >= exp
                    rhs_reduced = Rkbig(lift(rhs) / p^exp)
                    for j in 1:(p^(kbig - exp)-1)
                        if j % p != 0
                            c_start = mod(lift(rhs_reduced / j), p^(kbig-exp))
                            if !adder(p, n, kbig, lift_dict, a, p^exp * j, c_start, exp, Rn, Rkbig)
                                break
                            end
                        end
                    end
                end
            else
                for j in 1:(p^(kbig-exp)-1)
                    if j % p != 0 && !adder(p, n, kbig, lift_dict, a, p^exp * j, 0, exp, Rn, Rkbig)
                        break
                    end
                end
            end
        end
        println(length(lift_dict))
    end

    for a in 0:(p^kbig - 1)
        rhs = Rkbig(a * (trace - a) - det)
        if !iszero(rhs)
            pval = valuation(lift(rhs), p)
            for exp in n:pval
                rhs_reduced = Rkbig(lift(rhs) / p^exp)
                for j in 1:(p^(kbig - exp)-1)
                    if j % p != 0
                        c_start = mod(lift(rhs_reduced / j), p^(kbig-exp))
                        if !adder(p, n, kbig, lift_dict, a, p^exp*j, c_start, exp, Rn, Rkbig)
                            break
                        end
                    end
                end
            end
        else
            adder(p, n, kbig, lift_dict, a, 0, 0, kbig, Rn, Rkbig)
        end
    end
    println(length(lift_dict))
    println(length(values(lift_dict)))
    return values(lift_dict) # return only the lifted matrices (in p^kbig)
end

build_matrices_ratio2 (generic function with 1 method)

In [45]:
@time res = build_matrices_ratio2(3, 10, 6, 11, 10)
println(length(res))

393660
476766
484056
503982
523746
533628
533628
  0.713351 seconds (2.64 M allocations: 218.133 MiB, 38.85% gc time)
533628


In [6]:
# given a set of vectors, try to find a matrix A with minimum vp(det(A)) by trying the linear combination of the vectors
function det_check(p, ker, Rkbig)
    minVal = 100 # just an upper bound, any large number smaller than vp(discriminant) works
    l = size(ker, 2)
    ker_cols = [(Rkbig(ker[1, j]), Rkbig(ker[2, j]), Rkbig(ker[3, j]), Rkbig(ker[4, j])) for j in 1:l]
    for i in 1:l
        (a1, a2, a3, a4) = ker_cols[i]
        det = a1*a4 - a2*a3
        if det != 0 && valuation(lift(det), p) < minVal
            minVal = valuation(lift(det), p)
        end
        for j in (i+1):l
            b1, b2, b3, b4 = ker_cols[j]
            det = (a1+b1)*(a4+b4) - (a2+b2)*(a3+b3)
            if det != 0 && valuation(lift(det), p) < minVal
                minVal = valuation(lift(det), p)
            end
        end
    end
    return minVal
end

det_check (generic function with 1 method)

In [26]:
function ratio4(p, kbig, equations_template, matrices, max_e)
    Rkbig, = residue_ring(ZZ, p^kbig)
    count = Vector{Int}(undef, max_e+1)
    for i in 1:max_e+1
        count[i] = 0
    end
    println(length(matrices))
    for M in matrices
        equations = equations_template[M[1]]
        equations[1,3], equations[3,4], equations[2,1], equations[4,2] = M[2], M[2], M[3], M[3]
        ker = kernel(equations, side=:right)
        res = det_check(p, map(lift, ker), Rkbig)
        for e in max_e:-1:res
            count[e+1] += 1
        end
    end
    return count
end

ratio4 (generic function with 1 method)

In [27]:
function compute_ratio4(p, n, baseM, kbig, max_e)
    trace = baseM[1]+baseM[4]
    determinant = baseM[1]*baseM[4] - baseM[2]*baseM[3]    
    matrices = build_matrices_ratio2(p, kbig, n, trace, determinant)

    Rkbig, = residue_ring(ZZ, p^kbig)
    templates = Dict{zzModRingElem, zzModMatrix}()
    for M in matrices
        a = M[1]
        if !haskey(templates, a)
            templates[a] = matrix(Rkbig, [
                [a - baseM[1], -baseM[3], 0, 0],
                [0, 0, trace - a - baseM[1], -baseM[3]],
                [-baseM[2], a - baseM[4], 0, 0],
                [0, 0, -baseM[2], trace - a - baseM[4]]
            ])
        end
    end
    ratio4_val = ratio4(p, kbig, templates, matrices, max_e)
    
    ratio4_res = [rationalize(ratio4_val[e] / sl2size(p, n)) for e in 1:max_e+1]
    println("ratio 4:", ratio4_res)
end

compute_ratio4 (generic function with 2 methods)

In [28]:
@time compute_ratio4(3, 6, (1, 0, 0, 10), 10, 8)

533588
533588
ratio 4:Rational{Int64}[88//59049, 479//13122, 81917//78732, 263725//236196, 133397//118098, 133397//118098, 133397//118098, 133397//118098, 133397//118098]
  5.808630 seconds (34.65 M allocations: 2.319 GiB, 40.64% gc time)
